# L6: Multi-agent Collaboration for Financial Analysis

In this lesson, you will learn ways for making agents collaborate with each other.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM

In [2]:
from crewai import Agent, Task, Crew

**Note**: 
- The video uses `gpt-4-turbo`, but due to certain constraints, and in order to offer this course for free to everyone, the code you'll run here will use `gpt-3.5-turbo`.
- You can use `gpt-4-turbo` when you run the notebook _locally_ (using `gpt-4-turbo` will not work on the platform)
- Thank you for your understanding!

In [3]:
import os
from dotenv import load_dotenv
load_dotenv("F:\Social AI\multiagent-framework\.venv\.env")
os.environ["OPENAI_MODEL_NAME"] = 'gpt-4o-mini'
# SERPER_API_KEY = in .env

## crewAI Tools

In [4]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

## Creating Agents

In [5]:
data_analyst_agent = Agent(
    role="Data Analyst",
    goal="Monitor and analyze market data in real-time "
         "to identify trends and predict market movements.",
    backstory="Specializing in financial markets, this agent "
              "uses statistical modeling and machine learning "
              "to provide crucial insights. With a knack for data, "
              "the Data Analyst Agent is the cornerstone for "
              "informing trading decisions.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [6]:
trading_strategy_agent = Agent(
    role="Trading Strategy Developer",
    goal="Develop and test various trading strategies based "
         "on insights from the Data Analyst Agent.",
    backstory="Equipped with a deep understanding of financial "
              "markets and quantitative analysis, this agent "
              "devises and refines trading strategies. It evaluates "
              "the performance of different approaches to determine "
              "the most profitable and risk-averse options.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [7]:
execution_agent = Agent(
    role="Trade Advisor",
    goal="Suggest optimal trade execution strategies "
         "based on approved trading strategies.",
    backstory="This agent specializes in analyzing the timing, price, "
              "and logistical details of potential trades. By evaluating "
              "these factors, it provides well-founded suggestions for "
              "when and how trades should be executed to maximize "
              "efficiency and adherence to strategy.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [8]:
risk_management_agent = Agent(
    role="Risk Advisor",
    goal="Evaluate and provide insights on the risks "
         "associated with potential trading activities.",
    backstory="Armed with a deep understanding of risk assessment models "
              "and market dynamics, this agent scrutinizes the potential "
              "risks of proposed trades. It offers a detailed analysis of "
              "risk exposure and suggests safeguards to ensure that "
              "trading activities align with the firm’s risk tolerance.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

## Creating Tasks

In [9]:
# Task for Data Analyst Agent: Analyze Market Data
data_analysis_task = Task(
    description=(
        "Continuously monitor and analyze market data for "
        "the selected stock ({stock_selection}). "
        "Use statistical modeling and machine learning to "
        "identify trends and predict market movements."
    ),
    expected_output=(
        "Insights and alerts about significant market "
        "opportunities or threats for {stock_selection}."
    ),
    agent=data_analyst_agent,
)

In [10]:
# Task for Trading Strategy Agent: Develop Trading Strategies
strategy_development_task = Task(
    description=(
        "Develop and refine trading strategies based on "
        "the insights from the Data Analyst and "
        "user-defined risk tolerance ({risk_tolerance}). "
        "Consider trading preferences ({trading_strategy_preference})."
    ),
    expected_output=(
        "A set of potential trading strategies for {stock_selection} "
        "that align with the user's risk tolerance."
    ),
    agent=trading_strategy_agent,
)


In [11]:
# Task for Trade Advisor Agent: Plan Trade Execution
execution_planning_task = Task(
    description=(
        "Analyze approved trading strategies to determine the "
        "best execution methods for {stock_selection}, "
        "considering current market conditions and optimal pricing."
    ),
    expected_output=(
        "Detailed execution plans suggesting how and when to "
        "execute trades for {stock_selection}."
    ),
    agent=execution_agent,
)


In [12]:
# Task for Risk Advisor Agent: Assess Trading Risks
risk_assessment_task = Task(
    description=(
        "Evaluate the risks associated with the proposed trading "
        "strategies and execution plans for {stock_selection}. "
        "Provide a detailed analysis of potential risks "
        "and suggest mitigation strategies."
    ),
    expected_output=(
        "A comprehensive risk analysis report detailing potential "
        "risks and mitigation recommendations for {stock_selection}."
    ),
    agent=risk_management_agent,
)

## Creating the Crew
- The `Process` class helps to delegate the workflow to the Agents (kind of like a Manager at work)
- In the example below, it will run this hierarchically.
- `manager_llm` lets you choose the "manager" LLM you want to use.

In [13]:
from crewai import Process
from langchain_openai import ChatOpenAI

# Define the crew with agents and tasks
financial_trading_crew = Crew(
    agents=[data_analyst_agent, 
            trading_strategy_agent, 
            execution_agent, 
            risk_management_agent],
    
    tasks=[data_analysis_task, 
           strategy_development_task, 
           execution_planning_task, 
           risk_assessment_task],
    
    manager_llm=ChatOpenAI(model="gpt-4o-mini", 
                           temperature=0.7),
    process=Process.hierarchical,
    verbose=True
)

## Running the Crew

- Set the inputs for the execution of the crew.

In [14]:
# Example data for kicking off the process
financial_trading_inputs = {
    'stock_selection': 'AAPL',
    'initial_capital': '100000',
    'risk_tolerance': 'Medium',
    'trading_strategy_preference': 'Day Trading',
    'news_impact_consideration': True
}

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [15]:
### this execution will take some time to run
result = financial_trading_crew.kickoff(inputs=financial_trading_inputs)

# Agent: Crew Manager
## Task: Continuously monitor and analyze market data for the selected stock (AAPL). Use statistical modeling and machine learning to identify trends and predict market movements.
 

I encountered an error while trying to use the tool. This was the error: 2 validation errors for DelegateWorkToolSchema
task
  Input should be a valid string [type=string_type, input_value={'description': 'The task...ements.', 'type': 'str'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.9/v/string_type
context
  Input should be a valid string [type=string_type, input_value={'description': 'We are f...e data.', 'type': 'str'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.9/v/string_type.
 Tool Delegate work to coworker accepts these inputs: Tool Name: Delegate work to coworker
Tool Arguments: {'task': {'description': 'The task to delegate', 'type': 'str'}, 'context': {'description': 'The context for the task', 'typ

In [18]:
import pandas as pd

costs = 0.150 * (financial_trading_crew.usage_metrics.prompt_tokens + financial_trading_crew.usage_metrics.completion_tokens) / 1_000_000
print(f"Total costs: ${costs:.4f}")

# Convert UsageMetrics instance to a DataFrame
df_usage_metrics = pd.DataFrame([financial_trading_crew.usage_metrics.dict()])
df_usage_metrics

Total costs: $0.0164


,total_tokens,prompt_tokens,cached_prompt_tokens,completion_tokens,successful_requests
0,109385,100951,40192,8434,32


- Display the final result as Markdown.

In [17]:
from IPython.display import Markdown
Markdown(result.raw)

**Comprehensive Risk Analysis Report for AAPL Trading Strategies**

1. **Momentum Trading**:
   - **Risk Exposure**: The reliance on continuous price movement can lead to risks if a sudden shift occurs due to market conditions or economic news, resulting in potential significant losses from reverted positions.
   - **Mitigation Strategies**:
     - Use technical indicators for entry and exit points.
     - Implement trailing stops to lock in profits and limit losses.
     - Stay updated on market news that might affect AAPL's momentum.

2. **Breakout Strategy**:
   - **Risk Exposure**: False breakouts can mislead traders, resulting in losses when prices reverse shortly after breaking key levels.
   - **Mitigation Strategies**:
     - Confirm breakouts with increased volume to ensure validity.
     - Set stop-loss orders just below breakout points to contain potential losses.
     - Employ additional confirmations such as moving averages or momentum indicators.

3. **Reversal Trading**:
   - **Risk Exposure**: Engaging in reversal strategies can be precarious, as it involves counteracting prevailing trends, which, if incorrectly predicted, could lead to large financial losses.
   - **Mitigation Strategies**:
     - Utilize divergence indicators like RSI or MACD for reversal signs.
     - Maintain strict risk management protocols, including lower position sizing and tight stop losses.
     - Analyze overarching market sentiments and trends to guide entry decisions.

4. **News Trading**:
   - **Risk Exposure**: Sensitivity to news events can create unpredictable market reactions. Trading based on news could lead to losses if the anticipated move does not materialize.
   - **Mitigation Strategies**:
     - Examine the previous impacts of similar news on AAPL to gauge possible effects.
     - Prepare for various outcomes with a clear trading plan post-news release.
     - Establish defined exit strategies to manage volatility during news events.

5. **Overall Risk Management**:
   - **General Risk Exposure**: The overall trading landscape can be affected by volatility across markets, which may exacerbate losses during downturns.
   - **Mitigation Strategies**:
     - Diversify strategies to balance the risk profile across different methodologies and timeframes.
     - Limit capital exposure per trade to safeguard against substantial losses.
     - Conduct periodic assessments of trading performance and adjust strategies as necessary.

Through diligent risk assessment and strategic mitigation approaches tailored for each trading framework, traders can safeguard their positions against adverse market fluctuations while optimizing potential returns on AAPL trades. This detailed analysis is essential for aligning trading activities with the organization's broader risk tolerance and financial objectives.